In [71]:
%reset -f
import tensorflow as tf

data = tf.keras.datasets.fashion_mnist

(training_images, training_labels), (test_images, test_labels) = data.load_data()

training_images  = training_images / 255.0
test_images = test_images / 255.0

# MNIST Sequential API

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                                    tf.keras.layers.Dense(128,activation='relu'),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Plot model

In [ ]:
# !pip install pydot
# !pip install pydotplus
# Instal graphviz2.38 windows version
# Install it for everyone not just for current user
# make path changes in the pydotplus file if path change in environment variables does not work
from tensorflow.python.keras.utils.vis_utils import plot_model
plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')

In [ ]:
model.fit(training_images, training_labels, epochs=1)

# MNIST Functional API

In [ ]:
input = tf.keras.layers.Input(shape=(28,28))
x = tf.keras.layers.Flatten()(input)
x = tf.keras.layers.Dense(128,activation='relu')(x)
pred = tf.keras.layers.Dense(10,activation='softmax')(x)

model = tf.keras.Model(inputs=input,outputs=pred)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# !pip install pydot
# !pip install pydotplus
# Instal graphviz2.38 windows version
# Install it for everyone not just for current user
# make path changes in the pydotplus file if path change in environment variables does not work
from tensorflow.python.keras.utils.vis_utils import plot_model
plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')

In [ ]:
model.fit(training_images, training_labels, epochs=1)

# Custom Activation, Layer, Model, Loss class

In [139]:
%reset -f
import tensorflow as tf
from tensorflow.keras import backend as K

data = tf.keras.datasets.fashion_mnist

(training_images, training_labels), (test_images, test_labels) = data.load_data()

training_images  = training_images / 255.0
test_images = test_images / 255.0

# Custom Activation

def my_relu(x):
    return K.maximum(-0.1, x)

# Custom Layer

class MyDense(tf.keras.layers.Layer):

    def __init__(self, units=32,activation=None):
        '''Initializes the instance attributes'''
        super(MyDense, self).__init__()
        self.units = units
        
        self.activation = tf.keras.activations.get(activation)
        

    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        # initialize the weights
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel",
                initial_value=w_init(shape=(input_shape[-1], self.units),
                                     dtype='float32'),
                trainable=True)

        # initialize the biases
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
                initial_value=b_init(shape=(self.units,), dtype='float32'),
                trainable=True)

    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        return self.activation(tf.matmul(inputs, self.w) + self.b)

    
# Custom Model Test

class MNISTNetworkTest(tf.keras.Model):
    def __init__(self,**kwargs):
        super(MNISTNetworkTest,self).__init__(**kwargs)
        self.flatten = tf.keras.layers.Flatten()
        self.Dense1 = MyDense(128)
        self.classification = MyDense(10,activation='softmax')
    def call(self,inputs):
        input_a = inputs
        x = self.flatten(input_a)
        x = self.Dense1(x)
        # Custom activation layer
        x = tf.keras.layers.Lambda(my_relu)(x)
        outputs = self.classification(x)
        
        return outputs

# Custom model

class MNISTNetwork(tf.keras.Model):
    def __init__(self,**kwargs):
        super(MNISTNetwork,self).__init__(**kwargs)
        self.flatten = tf.keras.layers.Flatten()
        self.Dense1 = tf.keras.layers.Dense(128,activation='relu')
        self.classification = tf.keras.layers.Dense(10,activation='softmax')
    def call(self,inputs):
        input_a = inputs
        x = self.flatten(input_a)
        x = self.Dense1(x)
        outputs = self.classification(x)
        
        return outputs

    
    
model = MNISTNetworkTest()      



# Implement a custom loss

'''
# Below code is not correct
# A simple idea on how it can be done


class My_sparse_categorical_crossentropy(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
        
    def call(self,true,pred):
        sce =  tf.keras.losses.sparse_categorical_crossentropy()
        return sce(true,pred).numpy()

'''
    

model.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])



history = model.fit(training_images, training_labels, epochs=1)

Train on 60000 samples
60000/60000 [==============================] - 22s 360us/sample - loss: 0.5059 - accuracy: 0.8219
